In [2]:
# import sqlalchemy as sql
import pandas as pd
import numpy as np
# import csv
# import json
# from jsonpath_ng.ext import parse 
import re
from pprint import pprint


# Stanza
import stanza

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
from spacy import displacy


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
awards_filepath = "src/files/220k_awards_by_directors.csv"
movies_filepath     = "src/files/AllMoviesDetailsCleaned.csv"
cast_filepath       = "src/files/AllMoviesCastingRaw.csv"

awards_df = pd.read_csv(awards_filepath, delimiter=",", na_values='none')
movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')
cast_df = pd.read_csv(cast_filepath, delimiter=";", na_values='none')




/tmp/ipykernel_35691/4140812894.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')


In [5]:
awards_df

,director_name,ceremony,year,category,outcome,original_language
0,Aki Kaurismäki,ACCEC Awards,2004.0,ACCEC Award,Won,fi
1,Aki Kaurismäki,Alliance of Women Film Journalists,2012.0,EDA Award,Nominated,fi
2,Aki Kaurismäki,"Amanda Awards, Norway",1999.0,Amanda,Nominated,fi
3,Aki Kaurismäki,Argentinean Film Critics Association Awards,2013.0,Silver Condor,Nominated,fi
4,Aki Kaurismäki,Argentinean Film Critics Association Awards,2004.0,Silver Condor,Nominated,fi
...,...,...,...,...,...,...
225670,Courtney Hoffman,Satellite Awards,2016.0,Satellite Award,Nominated,en
225671,Fanni Metelius,Berlin International Film Festival,2012.0,Crystal Bear,Nominated,sv
225672,Fanni Metelius,Guldbagge Awards,2015.0,Guldbagge,Nominated,sv
225673,Fanni Metelius,Uppsala International Short Film Festival,2012.0,Honorable Mention,Won,sv


In [6]:
movies_df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5


In [7]:
movies_filtered_df = movies_df.head(30)

In [32]:
movies_filtered_df['overview'].dropna(inplace=True)

/tmp/ipykernel_35691/3861472245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_filtered_df['overview'].dropna(inplace=True)


In [10]:
# Convert to list
data = movies_filtered_df.overview.values.tolist()

# Filters
# Remove speical characters - except for , and -
data = [re.sub('[^a-zA-Z0-9,@\- ]', '', str(sent)) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove empty strings and blank spaces

data = [ x for x in data if (len(x.strip()) > 0)]

# Convert everything to lower case

data = [ str(x).lower() for x in data]




In [11]:
data[1]

'while racing to a boxing match, frank, mike, john and rey get more than they bargained for a wrong turn lands them directly in the path of fallon, a vicious, wise-cracking drug lord after accidentally witnessing fallon murder a disloyal henchman, the four become his unwilling prey in a savage game of cat amp mouse as they are mercilessly stalked through the urban jungle in this taut suspense drama'

## Convert sentence to words

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0])

['its', 'ted', 'the', 'bellhops', 'first', 'night', 'on', 'the', 'joband', 'the', 'hotels', 'very', 'unusual', 'guests', 'are', 'about', 'to', 'place', 'him', 'in', 'some', 'outrageous', 'predicaments', 'it', 'seems', 'that', 'this', 'evenings', 'room', 'service', 'is', 'serving', 'up', 'one', 'unbelievable', 'happening', 'after', 'another']


## Remove Stopwords and Lemmatize

In [13]:
# Define functions for stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
nlp = stanza.Pipeline('en')

nlp("don't won't")

2022-12-06 19:43:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-06 19:43:17,401 : INFO : Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-06 19:43:19 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-06 19:43:19,877 : INFO : Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
|

[
  [
    {
      "id": 1,
      "text": "do",
      "lemma": "do",
      "upos": "AUX",
      "xpos": "VBP",
      "feats": "Mood=Imp|VerbForm=Fin",
      "head": 3,
      "deprel": "aux",
      "start_char": 0,
      "end_char": 2,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "n't",
      "lemma": "not",
      "upos": "PART",
      "xpos": "RB",
      "head": 3,
      "deprel": "advmod",
      "start_char": 2,
      "end_char": 5,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "wo",
      "lemma": "will",
      "upos": "AUX",
      "xpos": "MD",
      "feats": "VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "start_char": 6,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "n't",
      "lemma": "not",
      "upos": "PART",
      "xpos": "RB",
      "head": 3,
      "deprel": "advmod",
     

In [18]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python3 -m spacy download en

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['bellhop', 'first', 'night', 'joband', 'hotel', 'unusual', 'guest', 'place', 'outrageous', 'predicament', 'seem', 'room', 'service', 'serve', 'unbelievable', 'happen']


## Create the Dictionary and Corpus needed for Topic Modeling

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Frequency list   
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [21]:
id2word[10]

'room'

In [22]:
[(id2word[cp[0]], cp[1])  for cp in corpus[10]]

[('young', 1),
 ('girl', 1),
 ('driver', 1),
 ('ensure', 1),
 ('give', 1),
 ('humanity', 1),
 ('key', 1),
 ('part', 1),
 ('save', 1),
 ('survival', 1),
 ('task', 1),
 ('taxi', 1),
 ('unintentionally', 1)]

## Building the Topic Model
 - The coherence score is good for num_topics = 3

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.022*"love" + 0.015*"thing" + 0.015*"class" + 0.015*"city" + 0.015*"great" + 0.008*"take" + 0.008*"yet" + 0.008*"life" + 0.008*"surfer" + 0.008*"far"'), (1, '0.018*"daughter" + 0.018*"love" + 0.018*"decide" + 0.018*"vast" + 0.018*"concert" + 0.018*"pirate" + 0.018*"life" + 0.009*"girl" + 0.009*"rock" + 0.009*"counterpoint"'), (2, '0.020*"supply" + 0.020*"water" + 0.020*"town" + 0.020*"accidentally" + 0.020*"gigantic" + 0.020*"fugitive" + 0.020*"declare" + 0.020*"springfield" + 0.020*"encase" + 0.020*"pollute"'), (3, '0.017*"bill" + 0.017*"make" + 0.017*"life" + 0.009*"set" + 0.009*"saving" + 0.009*"list" + 0.009*"death" + 0.009*"employer" + 0.009*"shoot" + 0.009*"last"'), (4, '0.015*"tour" + 0.015*"country" + 0.015*"value" + 0.015*"officer" + 0.015*"set" + 0.015*"track" + 0.015*"pretend" + 0.015*"alive" + 0.015*"himmelman" + 0.015*"pia"'), (5, '0.030*"film" + 0.025*"take" + 0.019*"make" + 0.013*"life" + 0.013*"documentary" + 0.013*"original" + 0.013*"new" + 0.013*"megacitie" + 0

## Visualize the topic model

In [25]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)

/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [26]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.198998 -0.063773       1        1  22.300381
0     -0.100871 -0.164430       2        1  17.127514
3     -0.015089  0.080757       3        1  14.790459
1     -0.053393  0.043735       4        1  13.512026
8      0.001210  0.093433       5        1  11.173843
7     -0.018236  0.005493       6        1   7.514275
9     -0.022860  0.013788       7        1   6.414516
4      0.002693 -0.002540       8        1   4.993004
2      0.002136 -0.000714       9        1   2.063291
6      0.005411 -0.005750      10        1   0.110691, topic_info=          Term      Freq     Total Category  logprob  loglift
70        film  4.000000  4.000000  Default  30.0000  30.0000
186   daughter  2.000000  2.000000  Default  29.0000  29.0000
248       girl  1.000000  1.000000  Default  28.0000  28.0000
100       take  4.000000  4.000000  Default  27.0000  27.0000
175       love  4.000000  4.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
28         get  0.001367  2.260450  Topic10  -6.0912  -0.6042
106      world  0.001365  1.839231  Topic10  -6.0930  -0.3997
333        put  0.001364  1.620799  Topic10  -6.0940  -0.2743
245  different  0.001363  1.548997  Topic10  -6.0943  -0.2293
246  disappear  0.001362  1.549109  Topic10  -6.0954  -0.2304

[439 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
159       2  0.873598       ability
387       1  0.832444        accept
16        5  0.801933  accidentally
430       3  0.899931     apartment
195       1  0.610292        appear
...     ...       ...           ...
108       1  0.534067           yet
108       2  0.534067           yet
158       2  0.352502         young
158       3  0.352502         young
158       4  0.352502         young

[285 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 4, 2, 9, 8, 10, 5, 3, 7])

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.64163059585697

Coherence Score:  0.5927375923357424


# Keyword of topics

In [28]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"love" + 0.015*"thing" + 0.015*"class" + 0.015*"city" + 0.015*"great" '
  '+ 0.008*"take" + 0.008*"yet" + 0.008*"life" + 0.008*"surfer" + 0.008*"far"'),
 (1,
  '0.018*"daughter" + 0.018*"love" + 0.018*"decide" + 0.018*"vast" + '
  '0.018*"concert" + 0.018*"pirate" + 0.018*"life" + 0.009*"girl" + '
  '0.009*"rock" + 0.009*"counterpoint"'),
 (2,
  '0.020*"supply" + 0.020*"water" + 0.020*"town" + 0.020*"accidentally" + '
  '0.020*"gigantic" + 0.020*"fugitive" + 0.020*"declare" + 0.020*"springfield" '
  '+ 0.020*"encase" + 0.020*"pollute"'),
 (3,
  '0.017*"bill" + 0.017*"make" + 0.017*"life" + 0.009*"set" + 0.009*"saving" + '
  '0.009*"list" + 0.009*"death" + 0.009*"employer" + 0.009*"shoot" + '
  '0.009*"last"'),
 (4,
  '0.015*"tour" + 0.015*"country" + 0.015*"value" + 0.015*"officer" + '
  '0.015*"set" + 0.015*"track" + 0.015*"pretend" + 0.015*"alive" + '
  '0.015*"himmelman" + 0.015*"pia"'),
 (5,
  '0.030*"film" + 0.025*"take" + 0.019*"make" + 0.013*"life" + '
  '0.013*"do

In [29]:
lda_model[corpus][0]

([(0, 0.9782034)],
 [(0, [0]),
  (1, [0]),
  (2, [0]),
  (3, [0]),
  (4, [0]),
  (5, [0]),
  (6, [0]),
  (7, [0]),
  (8, [0]),
  (9, [0]),
  (10, [0]),
  (11, [0]),
  (12, [0]),
  (13, [0]),
  (14, [0]),
  (15, [0])],
 [(0, [(0, 0.9999741)]),
  (1, [(0, 0.99997413)]),
  (2, [(0, 0.99997413)]),
  (3, [(0, 0.99997413)]),
  (4, [(0, 0.99997413)]),
  (5, [(0, 0.99997413)]),
  (6, [(0, 0.99997413)]),
  (7, [(0, 0.99997413)]),
  (8, [(0, 0.99997413)]),
  (9, [(0, 0.99997413)]),
  (10, [(0, 0.99997413)]),
  (11, [(0, 0.9999741)]),
  (12, [(0, 0.99997413)]),
  (13, [(0, 0.99997413)]),
  (14, [(0, 0.99997413)]),
  (15, [(0, 0.99997413)])])

## Attach topics to movies DF

In [30]:
movies_filtered_df['topic'] = [lda_model[corpus][text][0][0][0] for text in range(len(movies_filtered_df['overview']))]

/tmp/ipykernel_35691/1793517637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_filtered_df['topic'] = [lda_model[corpus][text][0][0][0] for text in range(len(movies_filtered_df['overview']))]


In [31]:
movies_filtered_df

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number,topic
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1,0
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1,8
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5,5
6,11,11000000,Adventure|Action|Science Fiction,tt0076759,en,Star Wars,Princess Leia is captured and held hostage by ...,10.492614,Lucasfilm,United States of America,...,English,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.0,6168,2,1,1,7
7,12,94000000,Animation|Family,tt0266543,en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",9.915573,Pixar Animation Studios,United States of America,...,English,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,5531,1,1,1,0
8,13,55000000,Comedy|Drama|Romance,tt0109830,en,Forrest Gump,A man with a low IQ has accomplished great thi...,10.351236,Paramount Pictures,United States of America,...,English,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7204,1,1,1,0
9,14,15000000,Drama,tt0169547,en,American Beauty,"Lester Burnham, a depressed suburban father in...",8.191009,DreamWorks SKG,United States of America,...,English,Released,Look closer.,American Beauty,7.9,2994,2,1,1,1
10,15,839727,Mystery|Drama,tt0033467,en,Citizen Kane,"Newspaper magnate, Charles Foster Kane is take...",3.82689,RKO Radio Pictures,United States of America,...,English,Released,It's Terrific!,Citizen Kane,7.9,1110,2,1,1,5
11,16,12800000,Drama|Crime|Music,tt0168629,en,Dancer in the Dark,"Selma, a Czech immigrant on the verge of blind...",2.106217,Fine Line Features,Argentina,...,English,Released,You don't need eyes to see.,Dancer in the Dark,7.6,348,26,12,1,3
12,17,0,Horror|Thriller|Mystery,tt0411267,en,The Dark,Adèle and her daughter Sarah are traveling on ...,1.253999,Constantin Film,Germany,...,English,Released,One of the living for one of the dead.,The Dark,5.6,69,4,2,2,9


/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/dse203/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/Anaconda/anaconda3/envs/dse203/lib/